<a href="https://colab.research.google.com/github/SilvanusTurkson/Face-Reg/blob/main/new_face_recognition_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!pip install deepface opencv-python streamlit pillow
!apt install libgl1-mesa-glx
!pip install deepface opencv-python-headless streamlit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [25]:
import streamlit as st
import cv2
import numpy as np
from PIL import Image
import os
import tempfile
import logging
from deepface import DeepFace

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration
KNOWN_FACES_DB = "/content/drive/MyDrive/images"  # Use Google Drive path
RECOGNITION_THRESHOLD = 0.4
MODEL_NAME = "VGG-Face"
DETECTOR_BACKEND = "opencv"
FRAME_SKIP = 3  # Process every 3rd frame to reduce load

# Initialize session state
if 'known_faces' not in st.session_state:
    st.session_state.known_faces = {}
if 'capture' not in st.session_state:
    st.session_state.capture = None
if 'processing' not in st.session_state:
    st.session_state.processing = False
if 'frame_count' not in st.session_state:
    st.session_state.frame_count = 0

def setup():
    """Initialize the known faces database"""
    try:
        if not os.path.exists(KNOWN_FACES_DB):
            os.makedirs(KNOWN_FACES_DB)
            st.info(f"Created directory for known faces at: {KNOWN_FACES_DB}")
        else:
            for file in os.listdir(KNOWN_FACES_DB):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    identity = os.path.splitext(file)[0]
                    st.session_state.known_faces[identity] = os.path.join(KNOWN_FACES_DB, file)
    except Exception as e:
        logger.error(f"Setup error: {str(e)}")
        st.error(f"Failed to initialize database: {str(e)}")

def add_known_face(image, identity):
    """Add a new face to the known faces database"""
    try:
        img_path = os.path.join(KNOWN_FACES_DB, f"{identity}.jpg")
        image.save(img_path)

        # Verify the image contains a face
        face = DeepFace.detectFace(img_path, detector_backend=DETECTOR_BACKEND)
        st.session_state.known_faces[identity] = img_path
        st.success(f"Successfully added {identity} to known faces!")
        return True
    except Exception as e:
        logger.error(f"Add face error: {str(e)}")
        st.error(f"Error adding face: {str(e)}")
        if 'img_path' in locals() and os.path.exists(img_path):
            os.remove(img_path)
        return False

def recognize_face(img_array):
    """Recognize a face from the known faces database"""
    try:
        # Save temporary image for recognition
        with tempfile.NamedTemporaryFile(suffix='.jpg') as tmp_file:
            Image.fromarray(img_array).save(tmp_file.name)

            # Find the most similar face
            dfs = DeepFace.find(
                img_path=tmp_file.name,
                db_path=KNOWN_FACES_DB,
                model_name=MODEL_NAME,
                detector_backend=DETECTOR_BACKEND,
                enforce_detection=False,
                silent=True
            )

            if len(dfs) > 0 and len(dfs[0]) > 0:
                best_match = dfs[0].iloc[0]
                if best_match['distance'] < RECOGNITION_THRESHOLD:
                    identity = os.path.splitext(os.path.basename(best_match['identity']))[0]
                    return identity, best_match['distance']

        return "Unknown", 1.0
    except Exception as e:
        logger.error(f"Recognition error: {str(e)}")
        return "No face detected", 1.0

def main():
    st.title("🔍 Real-Time Facial Recognition System")
    setup()

    # Sidebar for management
    st.sidebar.header("Face Database")

    # Add new face
    with st.sidebar.expander("Add New Face"):
        new_face_name = st.text_input("Name")
        uploaded_file = st.file_uploader("Upload Image", type=['jpg', 'png', 'jpeg'])
        if uploaded_file and new_face_name:
            image = Image.open(uploaded_file)
            if st.button("Add Face"):
                add_known_face(image, new_face_name)

    # Display known faces
    if st.session_state.known_faces:
        st.sidebar.subheader("Registered Faces")
        for identity in sorted(st.session_state.known_faces.keys()):
            st.sidebar.text(f"• {identity}")

    # Main content - Real-time recognition
    st.header("Live Camera Recognition")
    run = st.checkbox('Start Camera', key='run_recognition')
    FRAME_WINDOW = st.image([])
    status_text = st.empty()

    if run:
        if st.session_state.capture is None:
            st.session_state.capture = cv2.VideoCapture(0)
            st.session_state.capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            st.session_state.capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        while run and st.session_state.capture.isOpened():
            ret, frame = st.session_state.capture.read()
            if not ret:
                st.error("Failed to capture video")
                break

            # Convert to RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Only process every FRAME_SKIP frames
            st.session_state.frame_count += 1
            if st.session_state.frame_count % FRAME_SKIP == 0 and not st.session_state.processing:
                st.session_state.processing = True

                try:
                    # Recognize face
                    identity, confidence = recognize_face(frame_rgb)
                    status_text.text(f"Status: {identity} (confidence: {1-confidence:.2f})")

                    # Display result on frame
                    cv2.putText(frame_rgb, f"{identity}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                    cv2.putText(frame_rgb, f"Confidence: {1-confidence:.2f}", (10, 60),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

                except Exception as e:
                    status_text.text(f"Status: Error - {str(e)}")

                st.session_state.processing = False

            FRAME_WINDOW.image(frame_rgb)

            # Check if user stopped the recognition
            if not run:
                break
    else:
        if st.session_state.capture is not None:
            st.session_state.capture.release()
            st.session_state.capture = None
        status_text.text("Status: Ready")

if __name__ == "__main__":
    main()

2025-03-24 17:00:44.221 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 17:00:44.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 17:00:44.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 17:00:44.229 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 17:00:44.232 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 17:00:44.233 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 17:00:44.234 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 17:00:44.235 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
